In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("tmdb_star.db")


### Query 1 - Top actors by movie appearances
This query returns the 10 actors who appear in the highest number of movies, ordered by frequency.

In [8]:
query1 = """
SELECT  
    dpc.name AS company,
    ROUND(AVG(fm.revenue - fm.budget), 0) AS avg_profit,
    COUNT(*) AS num_movies
FROM link_production_companies lpc
JOIN dim_production_companies dpc ON lpc.dim_id = dpc.id
JOIN fact_movies fm ON lpc.fact_id = fm.movie_id
WHERE fm.revenue IS NOT NULL AND fm.budget IS NOT NULL
GROUP BY dpc.name
HAVING num_movies > 3
ORDER BY avg_profit DESC
LIMIT 10;

"""

pd.read_sql_query(query1, conn)

,company,avg_profit,num_movies
0,Lightstorm Entertainment,979976357.0,5
1,Illumination Entertainment,674783773.0,5
2,Fuji Television Network,638441957.0,6
3,Hasbro Studios,630949874.0,5
4,Marvel Studios,597569283.0,13
5,WingNut Films,511672955.0,11
6,Pixar Animation Studios,469299549.0,16
7,Heyday Films,454942815.0,10
8,Syncopy,445616261.0,8
9,Blue Sky Studios,419701161.0,9


In [2]:
query1 = """
SELECT p.name AS actor_name, COUNT(lc.movie_id) AS movie_count
FROM link_cast lc
JOIN dim_persons p ON lc.person_id = p.person_id
GROUP BY p.name
ORDER BY movie_count DESC
LIMIT 10;
"""

pd.read_sql_query(query1, conn)

,actor_name,movie_count
0,Samuel L. Jackson,67
1,Robert De Niro,57
2,Bruce Willis,51
3,Matt Damon,48
4,Morgan Freeman,46
5,Steve Buscemi,43
6,Liam Neeson,41
7,Owen Wilson,40
8,Johnny Depp,40
9,Nicolas Cage,39


### Query 2 - Average revenue by kyword
This query shows the keywords with the highest average revenue of the related films, ordered by average revenue. It filters out keywords that are associated with fewer than 5 movies.

In [10]:
query2 = """
SELECT 
    dk.name AS keyword,
    ROUND(AVG(fm.revenue), 0) AS avg_revenue,
    COUNT(fm.movie_id) AS movie_count
FROM link_keywords lk
JOIN dim_keywords dk ON lk.dim_id = dk.id
JOIN fact_movies fm ON fm.movie_id = lk.fact_id
WHERE fm.revenue IS NOT NULL AND fm.revenue > 0
GROUP BY dk.name
HAVING movie_count > 5
ORDER BY avg_revenue DESC
LIMIT 15;


"""

pd.read_sql_query(query2, conn)


,keyword,avg_revenue,movie_count
0,broom,901843593.0,6
1,orcs,897963612.0,7
2,marvel cinematic universe,779876975.0,13
3,wizard,702459394.0,8
4,elves,656302146.0,10
5,society,646052260.0,6
6,alien planet,646026612.0,7
7,cutting the cord,609060932.0,6
8,imax,598463813.0,23
9,tyrannosaurus rex,574645189.0,6


### Query 3 - Top actors by average movie revenue (minimum 3 movies)  
This query calculates the average revenue of the movies each actor has appeared in, considering only those who acted in at least 3 films. It shows the top 10 actors ranked by average movie revenue.

In [4]:
query3 = """
SELECT 
    p.name AS actor_name,
    COUNT(lc.movie_id) AS movie_count,
    AVG(f.revenue) AS average_revenue
FROM link_cast lc
JOIN dim_persons p ON lc.person_id = p.person_id
JOIN fact_movies f ON lc.movie_id = f.movie_id
GROUP BY p.person_id
HAVING movie_count >= 3
ORDER BY average_revenue DESC
LIMIT 10;
"""

pd.read_sql_query(query3, conn)


,actor_name,movie_count,average_revenue
0,Dileep Rao,3,1.234770e+09
1,Joseph O'Brien,3,1.165123e+09
2,Brian Schaeffer,4,1.099520e+09
3,Kevin Dorman,3,1.094323e+09
4,Jahnel Curfman,3,1.079832e+09
5,Julene Renee,5,1.034721e+09
6,Joti Nagra,3,1.024098e+09
7,Jon Lee Brody,3,9.927244e+08
8,Patrick Leahy,3,9.875859e+08
9,Dean O'Gorman,3,9.785078e+08


### Query 4 - Most versatile people (both actor and crew roles)
This query identifies people who worked as both actors and crew members, counting how many movies they did in each role.

In [5]:
query4 = """
SELECT 
    p.name,
    COUNT(DISTINCT mc.movie_id) AS actor_movies,
    COUNT(DISTINCT cr.movie_id) AS crew_movies
FROM dim_persons p
JOIN link_cast mc ON p.person_id = mc.person_id
JOIN link_crew cr ON p.person_id = cr.person_id
GROUP BY p.person_id
ORDER BY (actor_movies + crew_movies) DESC
LIMIT 10;
"""

pd.read_sql_query(query4, conn)

,name,actor_movies,crew_movies
0,Deborah Aquila,1,74
1,Steven Spielberg,6,65
2,Harvey Weinstein,2,68
3,Robert De Niro,57,12
4,Samuel L. Jackson,67,1
5,Danny Elfman,1,57
6,Stan Lee,27,31
7,Brad Pitt,38,17
8,Matt Damon,48,7
9,Bruce Willis,51,2


### Query 5 - Most frequent genre combinations in movies
This query finds the most common pairs of genres that appear together in the same movie.

In [6]:
query5 = """
SELECT 
    g1.name AS genre_1,
    g2.name AS genre_2,
    COUNT(*) AS pair_count
FROM link_genres lg1
JOIN link_genres lg2 ON lg1.fact_id = lg2.fact_id AND lg1.dim_id < lg2.dim_id
JOIN dim_genres g1 ON lg1.dim_id = g1.id
JOIN dim_genres g2 ON lg2.dim_id = g2.id
GROUP BY genre_1, genre_2
ORDER BY pair_count DESC
LIMIT 10;
"""

pd.read_sql_query(query5, conn)

,genre_1,genre_2,pair_count
0,Drama,Romance,603
1,Drama,Comedy,576
2,Drama,Thriller,554
3,Action,Thriller,547
4,Comedy,Romance,484
5,Adventure,Action,465
6,Thriller,Crime,414
7,Drama,Crime,381
8,Drama,Action,339
9,Comedy,Family,299


In [9]:
conn.close()
